# Training Machine Translation

In [1]:
import torch
from torch import nn
import d2l

## Loss Function

In [2]:
#@save
def sequence_mask(X, valid_len, value=0):
    """Mask irrelevant entries in sequences."""
    # `X` shape: (`batch_size`, `num_steps`)
    # [None, :] makes (`num_steps`,) to (1, `num_steps`)
    # [:, None] makes (`batch_size`) to (`batch_size`, 1)
    mask = torch.arange((X.size(1)), dtype=torch.float32,
                        device=X.device)[None, :] < valid_len[:, None]
    X[~mask] = value
    return X

In [3]:
#@save
class MaskedSoftmaxCELoss(nn.CrossEntropyLoss):
    """The softmax cross-entropy loss with masks."""
    # `pred` shape: (`batch_size`, `num_steps`, `vocab_size`)
    # `label` shape: (`batch_size`, `num_steps`)
    # `valid_len` shape: (`batch_size`,)
    def forward(self, pred, label, valid_len):
        weights = torch.ones_like(label)
        weights = sequence_mask(weights, valid_len)
        # 'none': no reduction will be applied, 'mean': the weighted mean of the output is taken
        self.reduction = 'none'
        # nn.CrossEntropyLoss((`batch_size`, `vocab_size`, `num_steps`), (`batch_size`, `num_steps`))
        unweighted_loss = super(MaskedSoftmaxCELoss, 
                                self).forward(pred.permute(0, 2, 1), label)
        # mean is taken on `num_steps`
        weighted_loss = (unweighted_loss * weights).mean(dim=1)
        return weighted_loss

## Training

Here we use the so called teacher forcing in decoder which trains much faster

In [4]:
#@save
def train_nmt(net, data_iter, lr, num_epochs, tgt_vocab):
    """e.g. english-french translation"""
    device = d2l.try_gpu()
    net.to(device)
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)
    loss = MaskedSoftmaxCELoss()
    net.train()  # dropout used, need specify
    animator = d2l.Animator(xlabel='epoch', ylabel='loss',
                            xlim=[10, num_epochs])
    for epoch in range(num_epochs):
        metric = d2l.Accumulator(2)  # Sum of training loss, no. of tokens
        for batch in data_iter:
            optimizer.zero_grad()
            X, X_valid_len, Y, Y_valid_len = [x.to(device) for x in batch]
            bos = torch.tensor([tgt_vocab['<bos>']] * Y.shape[0],
                               device=device).reshape(-1, 1)
            # teacher forcing decoder input
            dec_input = torch.cat([bos, Y[:, :-1]], 1)
            Y_hat, _ = net(X, dec_input, X_valid_len)
            l = loss(Y_hat, Y, Y_valid_len)
            l.sum().backward()
            optimizer.step()
            with torch.no_grad():
                metric.add(l.sum(), Y_valid_len.sum())
        if (epoch + 1) % 5 == 0:
            animator.add(epoch + 1, (metric[0] / metric[1],))

## Prediction

To predict the output sequence token by token, at each decoder time step the predicted token from the previous time step is fed into the decoder as an input.

![jupyter](../images/9/seq2seq-predict.svg)

In [5]:
#@save
def predict_nmt(net, src_sentence, src_vocab, tgt_vocab, num_steps, device=d2l.try_gpu()):
    """Predict given source sentence."""
    net.eval()
    # Encoder Preprocess
    src_tokens = src_vocab[src_sentence.lower().split(' ')] + [src_vocab['<eos>']]
    enc_valid_len = torch.tensor([len(src_tokens)], device=device)
    src_tokens = d2l.truncate_pad(src_tokens, num_steps, src_vocab['<pad>'])
    enc_X = torch.unsqueeze(
        torch.tensor(src_tokens, dtype=torch.long, device=device), dim=0)
    enc_outputs = net.encoder(enc_X, enc_valid_len)
    # Decoder first step Preprocess
    dec_state = net.decoder.init_state(enc_outputs, enc_valid_len)
    dec_X = torch.unsqueeze(
        torch.tensor([tgt_vocab['<bos>']], dtype=torch.long, device=device), dim=0)
    output_seq, attention_weight_seq = [], []
    for _ in range(num_steps):
        Y, dec_state = net.decoder(dec_X, dec_state)
        # We use the token with the highest prediction likelihood as the input
        # of the decoder at the next time step
        dec_X = Y.argmax(dim=2)
        pred = dec_X.squeeze(dim=0).type(torch.int32).item()
        # Once the end-of-sequence token is predicted, the generation of the
        # output sequence is complete
        if pred == tgt_vocab['<eos>']:
            break
        output_seq.append(pred)
    return ' '.join(tgt_vocab.to_tokens(output_seq))

## Evaluation

We can evaluate a predicted sequence by comparing it with the label sequence.

Denote by  $p_{n}$  the precision of  $n$-grams, which is the ratio of the number of matched  $n$-grams in the predicted and label sequences to the number of  $n$-grams in the predicted sequence.

Then, BLEU is defined as:

$$\exp\left(\min\left(0, 1 - \frac{\mathrm{len}_{\text{label}}}{\mathrm{len}_{\text{pred}}}\right)\right)\prod_{i=1}^{k}p_{n}^{1/{2^{n}}}$$

where  $k$  is the longest  $n$-grams for matching.

In [6]:
#@save
def bleu(pred_seq, label_seq, k):
    """Compute the BLEU."""
    pred_tokens, label_tokens = pred_seq.split(' '), label_seq.split(' ')
    len_pred, len_label = len(pred_tokens), len(label_tokens)
    score = math.exp(min(0, 1 - len_label / len_pred))
    for n in range(1, k + 1):
        num_matches, label_subs = 0, collections.defaultdict(int)
        for i in range(len_label - n + 1):
            label_subs[''.join(label_tokens[i:i + n])] += 1
        for i in range(len_pred - n + 1):
            if label_subs[''.join(pred_tokens[i:i + n])] > 0:
                num_matches += 1
                label_subs[''.join(pred_tokens[i:i + n])] -= 1
        score *= math.pow(num_matches / (len_pred - n + 1), math.pow(0.5, n))
    return score